# A Molecular Dynamics Simulation

In this tutorial, we will develop a simple molecular dynamics simulation for a Lennard-Jones fluid in step-by-step fashion. Once the simulation framework has been constructed, we will then use it to sample for configurations in order to examine how the radial distribution function for LJ fluid changes with temperature.

Recall the basic simulation scheme as described in class:

<img src="images/simulation.png" width="600px"/>

## 1. Importing packages

Before beginning, let us import the packages that we will be working with.
* [numpy](http://www.numpy.org/): NumPy is the fundamental package for scientific computing with Python.
* [matplotlib.pyplot](https://matplotlib.org/): a Python 2D plotting library which produces publication quality figures.
* [mpl_toolkits](https://matplotlib.org/mpl_toolkits/index.html): additional matplotlib toolkits. Mainly used in this notebook for 3D visualization.
* [matplotlib.animation](https://matplotlib.org/api/animation_api.html): a matplotlib animation library

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib.animation import FuncAnimation

## 2. Initialization

Unless the potential is of a simple form, we cannot sample starting positions exactly. A couple possible approaches:
* Start from some known configuration (e.g. from experiment)
* Randomly position particles, but we run into the risk of overlaps
* Initialize from lattice

For the purposes of this exercise, we will start from an ordered lattice. Complete the function below so that it returns an (num_dims x num_particles) numpy array such that the array holds the initial xyz coordinates of the particles in our system.

In [ ]:
def get_positions(num_dims, num_cell, num_particles, box_length):
    spacing = box_length / num_cell
    positions_1D = np.linspace(spacing, box_length, num_cell)
    if (num_dims == 3):
        r = np.meshgrid(positions_1D, positions_1D, positions_1D)
    elif (num_dims == 2):
        r = np.meshgrid(positions_1D, positions_1D)
    
    r = np.reshape(r, (num_dims, num_particles))
    # r = pbc(r, box_length)
    return r


r = get_positions(2, 5, 25, 10)
fig1 = plt.figure()
ax1 = fig1.add_subplot(111, aspect='equal', autoscale_on=False,
                    xlim=(0, 12), ylim=(0, 12))
pbcplot, = ax1.plot(r[0,:], r[1,:], 'bo', ms=5)

Additionally, we must also initialize the momenta for each particle in the system. Recall that from lecture, each of $p_x$, $p_y$, and $p_z$ can be drawn from a normal distributions such that

$$p_{avg} = 0, \sigma_p = \sqrt{mk_BT}$$

In filling in the below function for initializing momenta, remember that in reduced units, $k_B = 1$

In [ ]:
def get_momenta(num_dims, mass, temperature):
    num_particles = mass.shape[1]
    standard_dev = np.sqrt(temperature * mass) 
    mean = np.zeros(standard_dev.shape)
    return np.random.normal(mean, standard_dev, (num_dims, num_particles))

## 3. Interactions

For our simulation we will employ periodic boundary conditions in conjunction with the minimum image convention

<img src="images/pbc.png" width="300px"/>

The below function, given an unwrapped position or distance vector, will return the vector when taking into consideration pbc and minimum image convention albeit in a "hacky" way. Effectively, the function is enforcing that

$$
pbc(r_{x}) = 
\begin{cases}
    r_x, & \text{if } r_x < boxl/2\\
    r_x - boxl, & \text{otherwise}
\end{cases}
$$

for each component x, y, z of the vector. Note that the function as implemented implicitly assumes that the simulation box is centered on the origin.

In [ ]:
def pbc(vec, box_length):
    iboxl = 1.0 / box_length
    return vec - box_length * np.round(vec * iboxl)

For this simulation, we will be employing the Lennard-Jones model to describe the particle interactions. Recall, that the Lennard-Jones potential is a pairwise potential of the following form:

$$V(r) = 4\epsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^6\right]$$

Implement the function below such that it gives the LJ energy associated with two particles being $dist$ apart from one another (remember we are using reduced units!). Then verify that the potential looks as expected by running the code block

In [ ]:
def LJ(dist):
    dr2 = dist*dist
    idr2 = 1/dr2
    idr6 = idr2*idr2*idr2
    dv = 4 * (idr6*idr6 - idr6)
    df = 48 * dist * idr2 * (idr6*idr6 - 0.5*idr6)
    return dv, df


dists = np.linspace(0.5, 3.0, num=1000)
energies, forces = LJ(dists)

fig2 = plt.figure()
ax2 = fig2.add_subplot(111, aspect='equal', autoscale_on=True,
                      xlim=(0, 3.0), ylim=(-1.5, 1))
ljplot = ax2.plot(dists, energies)

Each iteration of the molecular dynamics simulation we will need to calculate each pairwise interaction in our system. Note that the function is vectorized for speed and makes use of Newton's 3rd law to avoid redundant force calculations.

In [ ]:
# Can speed up by vectorizing over all particles
def interactions(r, boxl):
    num_dims, num_particles = r.shape
    forces = np.zeros((num_dims, num_particles))
    potential = 0
    for pA in range(num_particles-1):
        rA = np.reshape(r[:, pA], (num_dims, 1))
        rB = r[:, pA+1:]
        
        dr = pbc(rA - rB, boxl)
        
        dist = np.linalg.norm(dr, axis=0)
        
        dv, df = LJ(dist)
        
        potential = potential + np.sum(dv)
        
        forces[:, pA] = forces[:, pA] + np.sum(dr*df/dist, axis=1)
        forces[:, pA+1:] = forces[:,pA+1:] - dr*df/dist
                
    return potential, forces

## 4. Main simulation loop

Below, we have the main simulation update function where we put everything together. To run a molecular dynamics simulation, we need to call the update() function in a loop. Note that we are running an NVT simulation and have couple a thermostat to the system.

In [ ]:
def update(frame, params, constants, file, visualize, ms):
    "simulation/animation update loop"
    global rect, ax, fig   # Probably should pass as args and not ref globals
    num_dims, temperature, dt, gamma, boxl, potE_0, kinE_0 = constants
    
    params['p']= params['p']+ 0.5*dt*params['forces']
    velocities = params['p']/ params['mass']
    
    # Evolve positions
    dr = dt*velocities
    params['r_noPBC'][:, :, frame+1] = params['r_noPBC'][:, :, frame] + dr
    params['r'] = pbc(params['r'] + dr, boxl)
    
    # Get potential energy and forces for new positions
    pe, params['forces'] = interactions(params['r'] , boxl)
    
    # Evolve momenta by dt/2
    params['p']= params['p']+ 0.5*dt*params['forces']
    
    # Thermostat the system: Anderson Thermostat
    if (np.random.uniform() < gamma*dt):
        params['p']= get_momenta(num_dims, params['mass'], temperature)
        
    # Print system properties
    ke = np.sum(np.square(params['p']) / (2 * params['mass']))
    tote = ke + pe
    T_obs = 2 * ke / (num_dims * num_particles)
    
    
    if frame % 10 == 0:
        paramstr = ("Time: %f, T: %3.4f, E: %10.4f, PE: %10.4f, KE: %10.4f\n" % 
              (frame*dt, T_obs, tote - (potE_0+kinE_0), pe-potE_0, ke-kinE_0))
        file.write(paramstr)
        
        
    if visualize:
        # update pieces of the animation
        if num_dims == 2:
            particles.set_data(params['r'][0,:], params['r'][1,:])
        elif num_dims == 3:
            particles.set_data(params['r'][0,:], params['r'][1,:])
            particles.set_3d_properties(params['r'][2:])
        particles.set_markersize(ms)
        return particles


## 5. Running the simulation

We need to specify certain parameters for our simulation including the density temperature of the system we want to simulate

In [ ]:
params = {}

# System parameters
# num_dims    = 2
# temperature = 35.0
# dt          = 0.0010
# gamma       = 100.0
# num_steps   = 2000
# ncell       = 7
# density     = 0.5
# visualize   = True
# ms          = 15

num_dims    = 3
temperature = 35.0
dt          = 0.0010
gamma       = 100.0
num_steps   = 2000
ncell       = 5
density     = 0.5
visualize   = True
ms          = 10

Some initialization are performed below

In [ ]:
# Compute box length from the density and number of particles
num_particles   = ncell**num_dims
volume          = num_particles / density
boxl            = volume**(1/num_dims)

# Initialize the mass for each particle
params['mass'] = np.ones((1, num_particles))

# Get initial positions of the particles, start on a square/cubic grid
params['r'] = get_positions(num_dims,ncell,num_particles,boxl);
params['r_noPBC']          = np.zeros((num_dims, num_particles, num_steps))
params['r_noPBC'][:, :, 0] = params['r']

# Get the initial momenta of the particles: start from the Boltzmann ditribution
params['p'] = get_momenta(num_dims, params['mass'], temperature)

# Calculate the initial potential energy and forces
potE, params['forces'] = interactions(params['r'], boxl)

potE_0 = potE
kinE_0 = np.sum(np.square(params['p'] / (2 * params['mass'])))

Here we set up the visualization environment and then run the molecular dynamics animation. Note that each frame of the animation is produced on-the-fly

In [ ]:
def init():
    """initialize animation"""
    if num_dims == 2:
        particles.set_data(params['r'][0,:], params['r'][1,:])
    elif num_dims == 3:
        particles.set_data(params['r'][0,:], params['r'][1,:])
        particles.set_3d_properties(params['r'][2:])
    return particles

constants = num_dims, temperature, dt, gamma, boxl, potE_0, kinE_0

# set up figure and animation
fig = plt.figure()
if num_dims == 2:
    ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                         xlim=(-0.5*boxl, 0.5*boxl), ylim=(-0.5*boxl, 0.5*boxl))
    particles, = ax.plot([], [], 'bo', ms=ms)
elif num_dims == 3:
    ax = p3.Axes3D(fig)
    ax.set_xlim3d([-0.5*boxl, 0.5*boxl])
    ax.set_ylim3d([-0.5*boxl, 0.5*boxl])
    ax.set_zlim3d([-0.5*boxl, 0.5*boxl])
    particles, = ax.plot([], [], [], 'ro', ms=ms)
    

# The animation loop doesn't play nice with I/O it seems. Be sure to
# run the next block to close the file after the simulation finishes.
# There's probably a better way but it might take some digging.
fname = 'simulation_params.out'
f = open(fname, 'w+')

# Below line runs simulation and animation. Alternatively if you want
# to just run simulation, then you can loop the update function.
anim = FuncAnimation(fig, update, frames=num_steps-1, 
                     interval=1, blit=True, init_func=init, 
                     repeat=False, fargs=(params, constants, f, visualize, ms))

Don't forget to run this after your simulation finishes to clean up some loose ends (in this case, an open file handle)

In [ ]:
f.close()